In [35]:
"""Baseline model. 1 layer LSTM with a classification layer trained for 100 epochs"""

'Baseline model. 1 layer LSTM with a classification layer trained for 100 epochs'

In [36]:
import sys
import os
import math

import numpy as np
import pandas as pd

In [37]:
df = pd.read_csv("../../data/processed/signalEUR_USD_Labelled_v1_processed.csv", index_col=0)

In [38]:
df.head(5)

,Date,Price,Open,High,Low,Change %,diff_1,label,sma,ema,cma,macd,macd_s,macd_h,roc,rsi,Bollinger_up,Bollinger_down,cci
0,2013-01-01,1.3205,1.3194,1.3220,1.3180,0.07%,NaN,NaN,NaN,1.320500,1.320500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2013-01-02,1.3186,1.3205,1.3302,1.3157,-0.14%,-0.0019,0.0,NaN,1.318956,1.319550,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN
2,2013-01-03,1.3048,1.3187,1.3192,1.3046,-1.05%,-0.0138,1.0,NaN,1.307931,1.314633,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN
3,2013-01-04,1.3069,1.3048,1.3091,1.2999,0.16%,0.0021,2.0,NaN,1.307136,1.312700,NaN,NaN,NaN,NaN,13.559663,NaN,NaN,NaN
4,2013-01-07,1.3117,1.3072,1.3121,1.3018,0.37%,0.0048,2.0,NaN,1.310649,1.312500,NaN,NaN,NaN,NaN,36.329386,NaN,NaN,NaN


In [39]:
import keras

In [40]:
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

def prepare_data(df, n=3):
    """
    n: number of days to window together 
    """
    df["label_3days"] = df["label"].shift(-3)
    df["label_5days"] = df["label"].shift(-5)
    df = df.dropna(axis=0)
    df = df.reset_index(drop=True)

    time = ["Date"]
    var = ["Price",  "sma", "ema", "cma", "macd",	"macd_s", 
               "macd_h", "roc", "rsi",	"Bollinger_up", "Bollinger_down", "cci"]
    columns = []
    for v in var:
        columns.append(v)
        for i in range(n - 1, 0, -1):
            columns.append(v + f"(t-{i})")
    
    labels = ["label", "label_3days", "label_5days"]
    features = df[time + columns + labels]

    X = features.drop(labels= time + labels, axis=1)
    print(X.head(5))
    Y = features[["label"]]

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    onehot_encoder = OneHotEncoder()
    Y = to_categorical(Y)
    
    return X, Y, features
    

In [55]:
from pandas import concat

def window(df, n=3):
    """
    Given time series data, window it such that we can get (t-n), (t-n-1) .. (t-1) and (t)
    """
    if n < 1:
        raise ValueError("n should be >= 1")
    n_var = df.shape[1]
    cols, names = list(), list()
    
    for i in range(n - 1, -1, -1):
        cols.append(df.shift(i))
        if i > 0:
            names += [(f"{j}(t-{i})") for j in list(df.columns)]
        else:
            names += [(f"{j}") for j in list(df.columns)]
    agg = concat(cols, axis=1)
    agg.columns = names
    agg.dropna(inplace=True)
    return agg

def prepare_and_window_data(df, n=3):
    df = window(df, n=n)
    return prepare_data(df, n=n)

def get_data_split(df, test_size=0.2, random_state=420, window_size=3):
    X, Y, features = prepare_and_window_data(df, n=window_size)
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=420)
    return X_train, X_test, Y_train, Y_test


In [52]:
from keras.models import Sequential
from keras.layers import Input
from keras.layers import LSTM
from keras.layers import Dense
from keras.losses import CategoricalCrossentropy

def build_model(X_dim, h_dim=100, num_classes=3):
    model = Sequential()
    model.add(LSTM(units=h_dim, input_shape=(X_dim, 1)))
    model.add(Dense(num_classes))
    model.compile(loss=CategoricalCrossentropy(from_logits=True), optimizer="adam", metrics=["accuracy"])
    model.summary()
    return model

In [53]:
from tqdm import tqdm
def run_experiment(df, start=1, end=10, epochs=100, batch_size=64):
    """
    Conduct an experiment to identify the optimal window size.
    start: Starting window size
    end: Ending window size
    Return: A dictionary containing validation accuracy for each window size
    """
    results = {}
    for cur_win_size in tqdm(range(start, end + 1)):
        X_train, X_test, Y_train, Y_test = get_data_split(df, window_size=cur_win_size)
        model = build_model(X_dim=X_train.shape[1])
        hist = model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=epochs, batch_size=batch_size)
        results[cur_win_size] = hist.history['accuracy'][-1]
    return results

In [54]:
results = run_experiment(df)
results

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

    Price       sma       ema       cma      macd    macd_s    macd_h  \
0  1.3363  1.344735  1.336730  1.335315  0.002701  0.005461 -0.002760   
1  1.3352  1.344925  1.335553  1.335311  0.001906  0.004750 -0.002844   
2  1.3388  1.345260  1.338051  1.335408  0.001549  0.004109 -0.002561   
3  1.3281  1.345075  1.330396  1.335211  0.000398  0.003367 -0.002969   
4  1.3189  1.344140  1.321553  1.334782 -0.001242  0.002445 -0.003688   

        roc        rsi  Bollinger_up  Bollinger_down         cci  
0 -0.000150  39.939646      1.364281        1.325189  -62.849696  
1 -0.004102  38.588551      1.363994        1.325856  -68.561252  
2 -0.004906  45.544851      1.363601        1.326919  -57.039491  
3 -0.012785  32.802173      1.364052        1.326098  -90.131403  
4 -0.012873  25.675429      1.366251        1.322029 -149.263447  
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm

Epoch 48/100
29/29 [==============================] - 0s 9ms/step - loss: 0.8636 - accuracy: 0.5687 - val_loss: 0.8806 - val_accuracy: 0.5832
Epoch 49/100
29/29 [==============================] - 0s 10ms/step - loss: 0.8616 - accuracy: 0.5833 - val_loss: 0.8856 - val_accuracy: 0.5637
Epoch 50/100
29/29 [==============================] - 0s 9ms/step - loss: 0.8485 - accuracy: 0.5871 - val_loss: 0.8716 - val_accuracy: 0.5788
Epoch 51/100
29/29 [==============================] - 0s 8ms/step - loss: 0.8401 - accuracy: 0.6066 - val_loss: 0.8633 - val_accuracy: 0.5875
Epoch 52/100
29/29 [==============================] - 0s 8ms/step - loss: 0.8381 - accuracy: 0.5925 - val_loss: 0.8811 - val_accuracy: 0.5637
Epoch 53/100
29/29 [==============================] - 0s 8ms/step - loss: 0.8413 - accuracy: 0.5828 - val_loss: 0.8780 - val_accuracy: 0.5940
Epoch 54/100
29/29 [==============================] - 0s 9ms/step - loss: 0.8336 - accuracy: 0.6001 - val_loss: 0.8785 - val_accuracy: 0.5983
Epoch

 10%|████████▎                                                                          | 1/10 [00:27<04:03, 27.08s/it]

    Price  Price(t-1)       sma  sma(t-1)       ema  ema(t-1)       cma  \
0  1.3352      1.3363  1.344925  1.344735  1.335553  1.336730  1.335311   
1  1.3388      1.3352  1.345260  1.344925  1.338051  1.335553  1.335408   
2  1.3281      1.3388  1.345075  1.345260  1.330396  1.338051  1.335211   
3  1.3189      1.3281  1.344140  1.345075  1.321553  1.330396  1.334782   
4  1.3190      1.3189  1.342780  1.344140  1.319589  1.321553  1.334377   

   cma(t-1)      macd  macd(t-1)    macd_s  macd_s(t-1)    macd_h  \
0  1.335315  0.001906   0.002701  0.004750     0.005461 -0.002844   
1  1.335311  0.001549   0.001906  0.004109     0.004750 -0.002561   
2  1.335408  0.000398   0.001549  0.003367     0.004109 -0.002969   
3  1.335211 -0.001242   0.000398  0.002445     0.003367 -0.003688   
4  1.334782 -0.002505  -0.001242  0.001455     0.002445 -0.003960   

   macd_h(t-1)       roc  roc(t-1)        rsi   rsi(t-1)  Bollinger_up  \
0    -0.002760 -0.004102 -0.000150  38.588551  39.939646    

...

    Price  Price(t-9)  Price(t-8)  Price(t-7)  Price(t-6)  Price(t-5)  \
0  1.3056      1.3363      1.3352      1.3388      1.3281      1.3189   
1  1.3022      1.3352      1.3388      1.3281      1.3189      1.3190   
2  1.3026      1.3388      1.3281      1.3189      1.3190      1.3062   
3  1.3051      1.3281      1.3189      1.3190      1.3062      1.3062   
4  1.2967      1.3189      1.3190      1.3062      1.3062      1.3139   

   Price(t-4)  Price(t-3)  Price(t-2)  Price(t-1)       sma  sma(t-9)  \
0      1.3190      1.3062      1.3062      1.3139  1.333895  1.344735   
1      1.3062      1.3062      1.3139      1.3056  1.330805  1.344925   
2      1.3062      1.3139      1.3056      1.3022  1.328365  1.345260   
3      1.3139      1.3056      1.3022      1.3026  1.325700  1.345075   
4      1.3056      1.3022      1.3026      1.3051  1.322920  1.344140   

   sma(t-8)  sma(t-7)  sma(t-6)  sma(t-5)  sma(t-4)  sma(t-3)  sma(t-2)  \
0  1.344925  1.345260  1.345075  1.344140  1.34

_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_11 (LSTM)              (None, 100)               40800     
                                                                 
 dense_11 (Dense)            (None, 3)                 303       
                                                                 
Total params: 41,103
Trainable params: 41,103
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
29/29 [==============================] - 4s 90ms/step - loss: 1.1002 - accuracy: 0.3210 - val_loss: 1.1008 - val_accuracy: 0.3384
Epoch 2/100
29/29 [==============================] - 2s 83ms/step - loss: 1.0978 - accuracy: 0.3384 - val_loss: 1.0966 - val_accuracy: 0.3536
Epoch 3/100
29/29 [==============================] - 2s 83ms/step - loss: 1.0974 - accuracy: 0.3449 - val_loss: 1.0944 - val_accuracy: 0.3796
Epoch 4/100
29/29 [=======================

Epoch 54/100
29/29 [==============================] - 2s 81ms/step - loss: 1.0038 - accuracy: 0.4818 - val_loss: 1.0339 - val_accuracy: 0.4664
Epoch 55/100
29/29 [==============================] - 2s 82ms/step - loss: 1.0076 - accuracy: 0.4834 - val_loss: 1.0279 - val_accuracy: 0.4902
Epoch 56/100
29/29 [==============================] - 2s 82ms/step - loss: 0.9978 - accuracy: 0.4943 - val_loss: 1.0219 - val_accuracy: 0.4729
Epoch 57/100
29/29 [==============================] - 2s 82ms/step - loss: 1.0002 - accuracy: 0.4894 - val_loss: 1.0190 - val_accuracy: 0.4859
Epoch 58/100
29/29 [==============================] - 2s 82ms/step - loss: 1.0013 - accuracy: 0.4883 - val_loss: 1.0382 - val_accuracy: 0.4425
Epoch 59/100
29/29 [==============================] - 2s 82ms/step - loss: 0.9888 - accuracy: 0.4900 - val_loss: 1.0215 - val_accuracy: 0.4620
Epoch 60/100
29/29 [==============================] - 2s 83ms/step - loss: 0.9897 - accuracy: 0.5046 - val_loss: 1.0377 - val_accuracy: 0.4447

100%|█████████████████████████████████████████████████████████████████████████████████| 10/10 [22:33<00:00, 135.35s/it]


{1: 0.6607142686843872,
 2: 0.9247835278511047,
 3: 0.9615592956542969,
 4: 0.8071506023406982,
 5: 0.7078590989112854,
 6: 0.7071583271026611,
 7: 0.627440333366394,
 8: 0.7276180386543274,
 9: 0.6959826350212097,
 10: 0.6317219138145447}

In [7]:
# Without price as a factor

In [48]:
X = features.drop(labels= ["Date"] + ["label", "label_3days", "label_5days"] + ["Price"], axis=1)

scaler = StandardScaler()
X = scaler.fit_transform(X)
X

array([[ 1.75711290e+00,  1.74688028e+00,  1.85073459e+00, ...,
        -5.82267337e-01, -6.48549048e-01, -7.15253591e-01],
       [ 1.74482298e+00,  1.78712648e+00,  1.84866607e+00, ...,
        -9.65166121e-01, -7.14635687e-01, -5.81952273e-01],
       [ 1.78504452e+00,  1.66750584e+00,  1.83821161e+00, ...,
        -1.64936920e+00, -5.81320962e-01, -9.64810006e-01],
       ...,
       [-5.66798082e-01, -5.95224900e-01, -5.71009894e-01, ...,
        -3.10471530e-01, -1.79884284e-03, -1.28458797e-01],
       [-5.95846969e-01, -5.63922301e-01, -5.73302048e-01, ...,
         9.39908017e-01, -1.27781876e-01, -3.10185606e-01],
       [-5.64563552e-01, -5.15850452e-01, -5.76097357e-01, ...,
        -1.33441060e-01, -3.09526962e-01,  9.40059888e-01]])

In [49]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=420)
print(X_train.shape, X_test.shape)
print(Y_train.shape, Y_test.shape)

(1843, 35) (461, 35)
(1843, 3) (461, 3)


In [50]:
model = Sequential()
model.add(LSTM(units=h_dim, input_shape=(X_train.shape[1], 1)))
model.add(Dense(num_classes))
model.compile(loss=CategoricalCrossentropy(from_logits=True), optimizer="adam", metrics=["accuracy"])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 100)               40800     
                                                                 
 dense_3 (Dense)             (None, 3)                 303       
                                                                 
Total params: 41,103
Trainable params: 41,103
Non-trainable params: 0
_________________________________________________________________


In [51]:
epochs = 100

model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=epochs, batch_size=64)

Epoch 1/100
29/29 [==============================] - 2s 27ms/step - loss: 1.0996 - accuracy: 0.3435 - val_loss: 1.0978 - val_accuracy: 0.3341
Epoch 2/100
29/29 [==============================] - 1s 19ms/step - loss: 1.0965 - accuracy: 0.3397 - val_loss: 1.0961 - val_accuracy: 0.3275
Epoch 3/100
29/29 [==============================] - 1s 19ms/step - loss: 1.0943 - accuracy: 0.3576 - val_loss: 1.0927 - val_accuracy: 0.3449
Epoch 4/100
29/29 [==============================] - 1s 19ms/step - loss: 1.0906 - accuracy: 0.3570 - val_loss: 1.0894 - val_accuracy: 0.3818
Epoch 5/100
29/29 [==============================] - 1s 19ms/step - loss: 1.0869 - accuracy: 0.3874 - val_loss: 1.0890 - val_accuracy: 0.3623
Epoch 6/100
29/29 [==============================] - 1s 19ms/step - loss: 1.0833 - accuracy: 0.3928 - val_loss: 1.0843 - val_accuracy: 0.3948
Epoch 7/100
29/29 [==============================] - 1s 19ms/step - loss: 1.0792 - accuracy: 0.4059 - val_loss: 1.0787 - val_accuracy: 0.4121
Epoch 